## 尋找Negative

### 計算訓練資料的字句最大字數

In [78]:
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import GRU,SimpleRNN, Activation, Dense,Bidirectional
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report

maxlen = 0
word_freqs = collections.Counter()
num_recs = 0
data_path = 'C:/Users/Tingchun.TC.Hung/Desktop/Record/NLP/查詢序列/影評正負面模型/Sentiment1_training.txt'
with open(data_path,'r+', encoding='UTF-8') as f: #影評 NLTK Sentiment1_training   train_mood
    for line in f:
        #print(line)
        label, sentence = line.strip().split("\t")
        words = nltk.word_tokenize(sentence.lower())
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            word_freqs[word] += 1
        num_recs += 1
print('max_len ',maxlen)
print('nb_words ', len(word_freqs))

Using TensorFlow backend.


max_len  42
nb_words  2268


### 準備數據 onehot encoding

In [20]:
## 準備數據
MAX_FEATURES = 2000
MAX_SENTENCE_LENGTH = 40
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word_index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(MAX_FEATURES))}
word_index["PAD"] = 0
word_index["UNK"] = 1
index2word = {v:k for k, v in word_index.items()}
X = np.empty(num_recs,dtype=list)
y = np.zeros(num_recs)
i=0
data_path = 'C:/Users/Tingchun.TC.Hung/Desktop/Record/NLP/查詢序列/影評正負面模型/Sentiment1_training.txt'
# 讀取訓練資料，將每一單字以 dictionary 儲存
with open(data_path,'r+', encoding='UTF-8') as f: #Sentiment1_training.txt train_mood.txt
    for line in f:
        label, sentence = line.strip().split("\t")
        words = nltk.word_tokenize(sentence.lower())
        seqs = []
        for word in words:
            if word in word_index:
                seqs.append(word_index[word])
            else:
                seqs.append(word_index["UNK"])
        X[i] = seqs
        y[i] = int(label)
        i += 1
# 字句長度不足補空白        
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

In [76]:
Save_path = 'C:/Users/Tingchun.TC.Hung/Desktop/Record/NLP/查詢序列/影評正負面模型/模型/'

### 實測副函式

In [83]:
def mood(text,model_path,data_path):

    Text1=input(text)
#     print(Text1,end='')

    #print("\n predict ......")
    model= load_model(model_path)

    ## 探索數據分析(EDA)
    # 計算訓練資料的字句最大字數
    maxlen = 0
    word_freqs = collections.Counter()
    num_recs = 0
    with open(data_path,'r+', encoding='UTF-8') as f:
        for line in f:
            label, sentence = line.strip().split("\t")
            words = nltk.word_tokenize(sentence.lower())
            if len(words) > maxlen:
                maxlen = len(words)
            for word in words:
                word_freqs[word] += 1
            num_recs += 1

    ## 準備數據
    MAX_FEATURES = 2000
    MAX_SENTENCE_LENGTH = 40
    vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
    word_index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(MAX_FEATURES))}
    word_index["PAD"] = 0
    word_index["UNK"] = 1
    index2word = {v:k for k, v in word_index.items()}
    X = np.empty(num_recs,dtype=list)
    y = np.zeros(num_recs)
    i=0
    # 讀取訓練資料，將每一單字以 dictionary 儲存
    with open(data_path,'r+', encoding='UTF-8') as f:
        for line in f:
            label, sentence = line.strip().split("\t")
            words = nltk.word_tokenize(sentence.lower())
            seqs = []
            for word in words:
                if word in word_index:
                    seqs.append(word_index[word])
                else:
                    seqs.append(word_index["UNK"])
            X[i] = seqs
            y[i] = int(label)
            i += 1

    # 字句長度不足補空白        
    X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)


    INPUT_SENTENCES = [Text1]# ,Text2,Text3
    XX = np.empty(len(INPUT_SENTENCES),dtype=list)
    # 轉換文字為數值
    i=0
    for sentence in  INPUT_SENTENCES:
        words = nltk.word_tokenize(sentence.lower())
        seq = []
        for word in words:
            if word in word_index:
                seq.append(word_index[word])
            else:
                seq.append(word_index['UNK'])
        XX[i] = seq
        i+=1

    XX = sequence.pad_sequences(XX, maxlen=MAX_SENTENCE_LENGTH)
    # 預測，並將結果四捨五入，轉換為 0 或 1
    labels = [int(round(x[0])) for x in model.predict(XX) ]
    label2word = {1:'正面', 0:'負面'}
    # 顯示結果

    #print("\n以下分析剛剛對話情緒:")

    for i in range(len(INPUT_SENTENCES)):
        print(''+label2word[labels[i]]) #, INPUT_SENTENCES[i]

In [14]:
# 資料劃分訓練組及測試組
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10

### RNN

In [23]:
# 模型構建

model = Sequential()
# 加『嵌入』層
model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
# 加『LSTM』層
model.add(SimpleRNN(batch_input_shape=(None, word_freqs), units= 50,unroll=True,)) 
model.add(Dense(1))
model.add(Activation("sigmoid"))
# binary_crossentropy:二分法
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

# 模型訓練
model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

# 預測
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
predictions = model.predict_classes(Xtest)

print("\n accuracy: %.3f" % (acc))
print()
print(classification_report(ytest, predictions))

print('{}   {}      {}'.format('預測','真實','句子'))
for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,MAX_SENTENCE_LENGTH)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
    print(' {}      {}     {}'.format(int(round(ypred)), int(ylabel), sent))
    

model.save(Save_path+"Mood_RNN_twitter.h5") 

Train on 5668 samples, validate on 1418 samples
Epoch 1/10
5668/5668 [==============================] - 2s 266us/step - loss: 0.2107 - acc: 0.9086 - val_loss: 0.1122 - val_acc: 0.9478
Epoch 2/10
5668/5668 [==============================] - 1s 175us/step - loss: 0.0308 - acc: 0.9903 - val_loss: 0.0885 - val_acc: 0.9654
Epoch 3/10
5668/5668 [==============================] - 1s 161us/step - loss: 0.0105 - acc: 0.9988 - val_loss: 0.0728 - val_acc: 0.9746
Epoch 4/10
5668/5668 [==============================] - 1s 167us/step - loss: 0.0023 - acc: 0.9996 - val_loss: 0.0739 - val_acc: 0.9774
Epoch 5/10
5668/5668 [==============================] - 1s 165us/step - loss: 0.0027 - acc: 0.9996 - val_loss: 0.0878 - val_acc: 0.9711
Epoch 6/10
5668/5668 [==============================] - 1s 169us/step - loss: 0.0013 - acc: 0.9996 - val_loss: 0.0720 - val_acc: 0.9788
Epoch 7/10
5668/5668 [==============================] - 1s 164us/step - loss: 9.9686e-04 - acc: 0.9998 - val_loss: 0.0880 - val_acc: 0.9

In [25]:
print("\n accuracy: %.3f" % (acc))
print()
print(classification_report(ytest, predictions))

print('{}   {}      {}'.format('預測','真實','句子'))
for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,MAX_SENTENCE_LENGTH)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
    print(' {}      {}     {}'.format(int(round(ypred)), int(ylabel), sent))


 accuracy: 0.977

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.97       626
         1.0       0.98      0.98      0.98       792

    accuracy                           0.98      1418
   macro avg       0.98      0.98      0.98      1418
weighted avg       0.98      0.98      0.98      1418

預測   真實      句子
 0      0     brokeback mountain was boring .
 0      0     always knows what i want , not guy crazy , hates harry potter ..
 0      0     my dad 's being stupid about brokeback mountain ...
 1      1     mission impossible 2 rocks ! ! ....
 1      1     the story of harry potter is a deep and profound one , and i love harry potter .


In [29]:
mood("Input",Save_path+"Mood_RNN_twitter.h5",data_path)

Input I am good


I am good  正面


### GRU

In [30]:
# 模型構建

model = Sequential()
# 加『嵌入』層
model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
# 加『LSTM』層
model.add(GRU(batch_input_shape=(None, word_freqs), units= 50,unroll=True,)) 
model.add(Dense(1))
model.add(Activation("sigmoid"))
# binary_crossentropy:二分法
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

# 模型訓練
model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

# 預測
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
predictions = model.predict_classes(Xtest)

print("\n accuracy: %.3f" % ( acc))
print()
print(classification_report(ytest, predictions))
for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,MAX_SENTENCE_LENGTH)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
    print(' {}      {}     {}'.format(int(round(ypred)), int(ylabel), sent))
    

model.save(Save_path+"Mood_GRU_twitter.h5") 

Train on 5668 samples, validate on 1418 samples
Epoch 1/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.1818 - acc: 0.9174 - val_loss: 0.0521 - val_acc: 0.9803
Epoch 2/10
5668/5668 [==============================] - 2s 386us/step - loss: 0.0151 - acc: 0.9959 - val_loss: 0.0431 - val_acc: 0.9880
Epoch 3/10
5668/5668 [==============================] - 2s 349us/step - loss: 0.0060 - acc: 0.9984 - val_loss: 0.0485 - val_acc: 0.9901
Epoch 4/10
5668/5668 [==============================] - 2s 362us/step - loss: 0.0033 - acc: 0.9996 - val_loss: 0.0459 - val_acc: 0.9880
Epoch 5/10
5668/5668 [==============================] - 2s 355us/step - loss: 0.0026 - acc: 0.9995 - val_loss: 0.0468 - val_acc: 0.9880
Epoch 6/10
5668/5668 [==============================] - 2s 352us/step - loss: 0.0048 - acc: 0.9993 - val_loss: 0.0511 - val_acc: 0.9901
Epoch 7/10
5668/5668 [==============================] - 2s 353us/step - loss: 0.0035 - acc: 0.9991 - val_loss: 0.0547 - val_acc: 0.9873
Ep

In [31]:
mood("Input",Save_path+"Mood_GRU_twitter.h5",data_path)

Input I hate u


I hate u  負面


### LSTM

In [32]:
# 模型構建

model = Sequential()
# 加『嵌入』層
model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
# 加『LSTM』層
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
# binary_crossentropy:二分法
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

# 模型訓練
model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

# 預測
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
predictions = model.predict_classes(Xtest)

print("\n accuracy: %.3f" % (acc))
print()
print(classification_report(ytest, predictions))
for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,MAX_SENTENCE_LENGTH)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
    print(' {}      {}     {}'.format(int(round(ypred)), int(ylabel), sent))
    

model.save(Save_path+"Mood_LSTM_twitter.h5") 


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 5668 samples, validate on 1418 samples
Epoch 1/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.2195 - acc: 0.9141 - val_loss: 0.0515 - val_acc: 0.9781
Epoch 2/10
5668/5668 [==============================] - 4s 690us/step - loss: 0.0205 - acc: 0.9936 - val_loss: 0.0512 - val_acc: 0.9859
Epoch 3/10
5668/5668 [==============================] - 4s 710us/step - loss: 0.0114 - acc: 0.9968 - val_loss: 0.0664 - val_acc: 0.9753
Epoch 4/10
5668/5668 [==============================] - 4s 699us/step - loss: 0.0056 - acc: 0.9977 - val_loss: 0.0432 - val_acc: 0.9894
Epoch 5/10
5668/5668 [==============================] - 4s 703us/step - loss: 0.0019 - acc: 0.9996 - val_loss: 0.0493 - val_acc: 0.9880
Epoch 6/10
5668/5668 [==============================] - 4s 701us/step - loss: 9.6567e-04 - acc: 0.9998 - val_loss: 0.0543 - val_acc: 0.9880
Epoch 7/10
5668/56

In [42]:
mood("Input",Save_path+"Mood_LSTM_twitter.h5",data_path)

Input Unmodified oligonucleotides containing the triplex target sequence were purchased from Gibco BRL (Gaithersburg, MD). Target duplexes were formed from a 1:1 mixture of unmodified complementary oligonucleotides in sterile water by denaturing at 80°C for 5 min and slow annealing at room temperature. The duplexes were phosphorylated with T4 polynucleotide kinase (Promega). Inserted sequences were as follows: pCAT-target and pCAT-AatII, AGTTTTGTGTCCC-CCTCTCAGGTGTCACAG; pCAT-Eco72I, AGTTTTGTG-TCCCCCTCTCACGTGTCACAG, where the single base alteration in the sequence (note the bold C) generates an Eco72I site. The pCAT-6target insert (216 bp long) was generated using two oligonucleotides purchased from Genosys (The Woodlands, TX). The first oligonucleotide (108 bp) contained three consecutive target sequences: GCAAGCTTAGTTTTG-TGTCCCCCTCTCAGGTGTCACAGAGTTTTGTGTCCCCC-TCTCAGGTGTCACAGAGTTTTGTGTCCCCCTCTCAGG-TGTCACAGGGATCCGGCG (target sequence AGTTTTGTGTCCCCCTCTCAGGTGTCACAG) and 18 bp of sequence

Unmodified oligonucleotides containing the triplex target sequence were purchased from Gibco BRL (Gaithersburg, MD). Target duplexes were formed from a 1:1 mixture of unmodified complementary oligonucleotides in sterile water by denaturing at 80°C for 5 min and slow annealing at room temperature. The duplexes were phosphorylated with T4 polynucleotide kinase (Promega). Inserted sequences were as follows: pCAT-target and pCAT-AatII, AGTTTTGTGTCCC-CCTCTCAGGTGTCACAG; pCAT-Eco72I, AGTTTTGTG-TCCCCCTCTCACGTGTCACAG, where the single base alteration in the sequence (note the bold C) generates an Eco72I site. The pCAT-6target insert (216 bp long) was generated using two oligonucleotides purchased from Genosys (The Woodlands, TX). The first oligonucleotide (108 bp) contained three consecutive target sequences: GCAAGCTTAGTTTTG-TGTCCCCCTCTCAGGTGTCACAGAGTTTTGTGTCCCCC-TCTCAGGTGTCACAGAGTTTTGTGTCCCCCTCTCAGG-TGTCACAGGGATCCGGCG (target sequence AGTTTTGTGTCCCCCTCTCAGGTGTCACAG) and 18 bp of sequence to al

### BILSTM

In [43]:
# 模型構建

model = Sequential()
# 加『嵌入』層
model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
# 加『LSTM』層
model.add(Bidirectional(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1))
model.add(Activation("sigmoid"))
# binary_crossentropy:二分法
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

# 模型訓練
model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

# 預測
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
predictions = model.predict_classes(Xtest)

print("\n accuracy: %.3f" % (acc))
print()
print(classification_report(ytest, predictions))
for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,MAX_SENTENCE_LENGTH)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
    print(' {}      {}     {}'.format(int(round(ypred)), int(ylabel), sent))
    

model.save(Save_path+"Mood_BiLSTM_twitter.h5") 

Train on 5668 samples, validate on 1418 samples
Epoch 1/10
5668/5668 [==============================] - 10s 2ms/step - loss: 0.2282 - acc: 0.8968 - val_loss: 0.0576 - val_acc: 0.9824
Epoch 2/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.0287 - acc: 0.9914 - val_loss: 0.0723 - val_acc: 0.9746
Epoch 3/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.0072 - acc: 0.9981 - val_loss: 0.0647 - val_acc: 0.9824
Epoch 4/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.0075 - acc: 0.9974 - val_loss: 0.0495 - val_acc: 0.9866
Epoch 5/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.0032 - acc: 0.9995 - val_loss: 0.0640 - val_acc: 0.9803
Epoch 6/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.0025 - acc: 0.9995 - val_loss: 0.0485 - val_acc: 0.9880
Epoch 7/10
5668/5668 [==============================] - 6s 1ms/step - loss: 0.0023 - acc: 0.9996 - val_loss: 0.0521 - val_acc: 0.9866
Epoch 8/10
56

In [84]:
mood("Input : ",Save_path+"Mood_BiLSTM_twitter.h5",data_path)

Input :  The pCAT-6target insert (216 bp long) was generated using two oligonucleotides purchased from Genosys (The Woodlands, TX). The first oligonucleotide (108 bp) contained three consecutive target sequences: GCAAGCTTAGTTTTG-TGTCCCCCTCTCAGGTGTCACAGAGTTTTGTGTCCCCC-TCTCAGGTGTCACAGAGTTTTGTGTCCCCCTCTCAGG-TGTCACAGGGATCCGGCG (target sequence AGTTTTGTGTCCCCCTCTCAGGTGTCACAG) and 18 bp of sequence to allow annealing to a second oligonucleotide.


負面
